# Compile boostvqe circuit
This notebook serves to run boostvqe and extract the transpiled circuit.

In [1]:
import numpy as np
import qibo
from qibo import hamiltonians
from qibo.backends import construct_backend
from qibo.quantum_info.metrics import fidelity
from boostvqe.ansatze import VQE, build_circuit
from boostvqe.models.dbi import double_bracket_evolution_oracles
from boostvqe.models.dbi.double_bracket_evolution_oracles import (
    FrameShiftedEvolutionOracle,
    IsingNNEvolutionOracle,
    MagneticFieldEvolutionOracle,
    XXZ_EvolutionOracle,
)
from boostvqe.models.dbi.group_commutator_iteration_transpiler import (
    DoubleBracketRotationType,
    GroupCommutatorIterationWithEvolutionOracles,
)
from boostvqe.utils import (
    OPTIMIZATION_FILE,
    PARAMS_FILE,
    build_circuit,
    optimize_D,
    select_recursion_step_gd_circuit,
)
import json
import time

In [2]:
def report(vqe, hamiltonian, gci, step, eo_d, mode):
    energies = hamiltonian.eigenvalues()
    ground_state_energy = float(energies[0])
    vqe_energy = float(hamiltonian.expectation(vqe.circuit().state()))
    gci_loss = float(gci.loss(step, eo_d, mode))
    gap = float(energies[1] - energies[0] + 1e-6)

    return (
        dict(
            nqubits=hamiltonian.nqubits,
            gci_loss=float(gci_loss),
            vqe_energy=float(vqe_energy),
            target_energy=ground_state_energy,
            diff_vqe_target=vqe_energy - ground_state_energy,
            diff_gci_target=gci_loss - ground_state_energy,
            gap=gap,
            diff_vqe_target_perc=abs(vqe_energy - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc=abs(gci_loss - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            fidelity_witness_vqe=1 - (vqe_energy - ground_state_energy) / gap,
            fidelity_witness_gci=1 - (gci_loss - ground_state_energy) / gap,
            fidelity_vqe=fidelity(vqe.circuit().state(), hamiltonian.ground_state()),
            fidelity_gci=fidelity(
                gci.get_composed_circuit()().state(), hamiltonian.ground_state()
            ),
        )
        | gci.get_gate_count_dict()
    )

def print_report(report: dict):
    print(
        f"\
    The target energy is {report['target_energy']}\n\
    The VQE energy is {report['vqe_energy']} \n\
    The DBQA energy is {report['gci_loss']}. \n\
    The difference is for VQE is {report['diff_vqe_target']} \n\
    and for the DBQA {report['diff_gci_target']} \n\
    which can be compared to the spectral gap {report['gap']}.\n\
    The relative difference is \n\
        - for VQE {report['diff_vqe_target_perc']}% \n\
        - for DBQA {report['diff_gci_target_perc']}%.\n\
    The energetic fidelity witness of the ground state is: \n\
        - for the VQE  {report['fidelity_witness_vqe']} \n\
        - for DBQA {report['fidelity_witness_gci']}\n\
    The true fidelity is \n\
        - for the VQE  {report['fidelity_vqe']}\n\
        - for DBQA {report['fidelity_gci']}\n\
                    "
    )
    print(
        f"The boosting circuit used {report['nmb_cnot']} CNOT gates coming from compiled XXZ evolution and {report['nmb_cz']} CZ gates from VQE.\n\
For {report['nqubits']} qubits this gives n_CNOT/n_qubits = {report['nmb_cnot_relative']} and n_CZ/n_qubits = {report['nmb_cz_relative']}"
    )

In [3]:
qibo.set_backend("numpy")
vqe_backend = construct_backend(backend="numpy")

[Qibo 0.2.11|INFO|2024-08-20 09:46:39]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In [4]:
nqubits = 3
nlayers = 3
epochs = 100
path = "./results/big_architectures_small_lr/sgd_3q_3l_42/"
OPTIMIZATION_METHOD="cma"
OPTIMIZATION_CONFIG="{ \"maxiter\": 2}"

In [5]:
# load data
opt_options = json.loads(OPTIMIZATION_CONFIG)

params = np.array(
    np.load(path + PARAMS_FILE, allow_pickle=True).tolist()[0][epochs]
)

In [6]:
hamiltonian = hamiltonians.XXZ(nqubits, delta=0.5, backend=vqe_backend)

In [7]:
vqe = VQE(
    build_circuit(
        nqubits=nqubits,
        nlayers=nlayers,
    ),
    hamiltonian=hamiltonian,
)
vqe.circuit.set_parameters(params)

In [8]:
print(vqe.circuit.draw())

q0: ─RY─RZ─o─RY─RZ─o─RY─RZ─o─RY─RZ─o─RY─RZ─o─RY─RZ─o─RY─
q1: ─RY─RZ─Z─RY─RZ─|─RY─RZ─Z─RY─RZ─|─RY─RZ─Z─RY─RZ─|─RY─
q2: ─RY─RZ───RY─RZ─Z─RY─RZ───RY─RZ─Z─RY─RZ───RY─RZ─Z─RY─


In [9]:
base_oracle = XXZ_EvolutionOracle.from_nqubits(
    nqubits=nqubits, delta=0.5, steps=2, order=2
)
# steps: DBI steps
# order: Trotter order

oracle = FrameShiftedEvolutionOracle.from_evolution_oracle(
    before_circuit=vqe.circuit.invert(),
    after_circuit=vqe.circuit,
    base_evolution_oracle=base_oracle,
)
db_rotation = DoubleBracketRotationType.group_commutator_third_order_reduced
gci = GroupCommutatorIterationWithEvolutionOracles(
        oracle,
        db_rotation
    )

In [10]:
eo_d_type = getattr(double_bracket_evolution_oracles, "IsingNNEvolutionOracle")
print(
        f"The gci mode is {gci.double_bracket_rotation_type} rotation with {eo_d_type.__name__} as the oracle.\n"
    )

The gci mode is DoubleBracketRotationType.group_commutator_third_order_reduced rotation with IsingNNEvolutionOracle as the oracle.



In [11]:
NSTEPS = 1
for gci_step_nmb in range(NSTEPS):
    print(f"Optimizing GCI step {gci_step_nmb+1} with optimizer {OPTIMIZATION_METHOD}")
    it = time.time()
    if OPTIMIZATION_METHOD == "sgd":
            params = (
                [4 - np.sin(x / 3) for x in range(nqubits)]
                if eo_d_type == MagneticFieldEvolutionOracle
                else [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
            )
            mode, best_s, best_b, eo_d = select_recursion_step_gd_circuit(
                gci,
                mode=db_rotation,
                eo_d_type=eo_d_type,
                params=params,
                step_grid=np.linspace(1e-5, 2e-2, 30),
                lr_range=(1e-3, 1),
                nmb_gd_epochs=opt_options["gd_epochs"],
                threshold=1e-4,
                max_eval_gd=30,
            )

            opt_dict = {"sgd_extras": "To be defined"}
    else:
        if gci_step_nmb == 0:
            p0 = [0.01]
            if eo_d_type == MagneticFieldEvolutionOracle:
                p0.extend([4 - np.sin(x / 3) for x in range(nqubits)])
            elif eo_d_type == IsingNNEvolutionOracle:
                p0.extend(
                    [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
                )

        else:
            p0 = [best_s]
            p0.extend(best_b)
        optimized_params, opt_dict = optimize_D(
            params=p0,
            gci=gci,
            eo_d_type=eo_d_type,
            mode=db_rotation,
            method=OPTIMIZATION_METHOD,
            **opt_options,
        )
        best_s = optimized_params[0]
        best_b = optimized_params[1:]
        eo_d = eo_d_type.load(best_b)
    print(hamiltonian.eigenvalues())
    print(f"Total optimization time required: {time.time() - it} seconds")
    gci.mode_double_bracket_rotation = db_rotation

    gci(best_s, eo_d, db_rotation)
    this_report = report(vqe, hamiltonian, gci, best_s, eo_d, db_rotation)
    print_report(this_report)



Optimizing GCI step 1 with optimizer cma
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=291920, Tue Aug 20 09:46:39 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 -2.498108001327918e+00 1.0e+00 4.51e-01  4e-01  5e-01 0:00.1
    2     18 -2.497846923454115e+00 1.2e+00 4.47e-01  4e-01  5e-01 0:00.3
termination on maxiter=2 (Tue Aug 20 09:46:40 2024)
final/bestever f-value = -2.499297e+00 -2.499297e+00 after 19/19 evaluations
incumbent solution: [0.011856302197761154, 4.447155615340837, 3.666578977632253, 3.4599779885891806, 1.2327812281917325, 1.554494203981013, 1.6152617826817992]
std deviation: [0.39911360661970197, 0.45792949155497215, 0.40084650105638736, 0.45024231416088206, 0.4320282836117505, 0.4650726921957328, 0.473864559571338]
[-2.5 -2.5 -2.5 -2.5  1.5  1.5  3.5  3.5]
Total optimization time required: 0.40068912506103516 seconds


ZeroDivisionError: float division by zero

In [ ]:
c = gci.get_composed_circuit(best_s, eo_d)
print(c.draw())

q0:     ─o────o──────────────────────X─RZ─X─RZ─o────o──────────────────────X─R ...
q1:     ─X─RZ─X─o────o───────────────|────|─RZ─X─RZ─X─o────o───────────────|── ...
q2:     ────────X─RZ─X─o────o────────|────|─RZ────────X─RZ─X─o────o────────|── ...
q3:     ───────────────X─RZ─X─o────o─|────|─RZ───────────────X─RZ─X─o────o─|── ...
q4:     ──────────────────────X─RZ─X─o────o─RZ──────────────────────X─RZ─X─o── ...

q0: ... Z─X─RZ─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─RZ─o───RY─RZ───o─RY─ ...
q1: ... ──|─RZ─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─RZ─Z───RY─RZ─o─|─RY─ ...
q2: ... ──|─RZ─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─RZ───o─RY─RZ─Z─|─RY─ ...
q3: ... ──|─RZ─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─RZ───Z─RY─RZ───|─RY─ ...
q4: ... ──o─RZ─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─RZ─────RY─RZ───Z─RY─ ...

q0: ... ─────────────────────RZ─o─RY─X─RY─o────X───RZ─o──────X───RZ─────────── ...
q1: ... ───X───RZ─o──────X───RZ─|────|────|─RZ─o───RY─X───RY─o─────────X───RZ─ ...
q2

## Transform into Qiskit circuit

In [ ]:
from qibo import gates, models
import qiskit.qasm2
def qibo_to_qiskit_circuit(circuit):
    qasm_code = models.Circuit.to_qasm(circuit)
    return qiskit.qasm2.loads(qasm_code)

In [ ]:
vqe_c = qibo_to_qiskit_circuit(vqe.circuit)
vqe_c.draw()

┌─────────────┐┌─────────────┐               ┌─────────────┐»
q_0: ┤ Ry(-1.0018) ├┤ Rz(-2.3789) ├───────■───────┤ Ry(-3.2155) ├»
     └┬────────────┤├─────────────┤       │       └┬────────────┤»
q_1: ─┤ Ry(3.1246) ├┤ Rz(-2.8363) ├───────■────────┤ Ry(2.7717) ├»
      ├────────────┤└┬────────────┤                ├────────────┤»
q_2: ─┤ Ry(1.8289) ├─┤ Rz(2.0549) ├───────■────────┤ Ry(2.2151) ├»
     ┌┴────────────┤┌┴────────────┤       │       ┌┴────────────┤»
q_3: ┤ Ry(0.92673) ├┤ Rz(0.91442) ├───────■───────┤ Ry(-2.0361) ├»
     ├─────────────┤└┬────────────┤┌─────────────┐└┬────────────┤»
q_4: ┤ Ry(-2.3098) ├─┤ Rz(1.2341) ├┤ Ry(-1.9534) ├─┤ Rz(-1.252) ├»
     └─────────────┘ └────────────┘└─────────────┘ └────────────┘»
«     ┌─────────────┐                   ┌─────────────┐  ┌────────────┐»
«q_0: ┤ Rz(-1.8666) ├─────────────────■─┤ Ry(0.75898) ├──┤ Rz(1.4941) ├»
«     ├─────────────┤                 │ ├─────────────┤ ┌┴────────────┤»
«q_1: ┤ Rz(-1.3928) ├────────■────────┼─┤ Ry(-1.9317) ├─┤ Rz(-2.1252) ├»
«     ├─────────────┤        │        │ ├─────────────┤ ├─────────────┤»
«q_2: ┤ Rz(0.42583) ├────────■────────┼─┤ Ry(-1.4245) ├─┤ Rz(0.28077) ├»
«     ├─────────────┴┐┌─────────────┐ │ ├─────────────┤ └─────────────┘»
«q_3: ┤ Rz(-0.65839) ├┤ Ry(-0.9817) ├─┼─┤ Rz(0.32141) ├────────────────»
«     └──────────────┘└─────────────┘ │ ├─────────────┴┐┌─────────────┐»
«q_4: ────────────────────────────────■─┤ Ry(-0.51244) ├┤ Rz(-2.8011) ├»
«                                       └──────────────┘└─────────────┘»
«                    ┌────────────┐  ┌────────────┐                 »
«q_0: ──────■────────┤ Ry(1.0889) ├──┤ Rz(1.6706) ├───────────────■─»
«           │       ┌┴────────────┤ ┌┴────────────┤               │ »
«q_1: ──────■───────┤ Ry(-1.9716) ├─┤ Rz(-1.4455) ├──────■────────┼─»
«                   ├─────────────┤ ├─────────────┤      │        │ »
«q_2: ──────■───────┤ Ry(-2.4942) ├─┤ Rz(-2.4033) ├──────■────────┼─»
«           │       └┬────────────┤ └┬────────────┤┌────────────┐ │ »
«q_3: ──────■────────┤ Ry(2.6148) ├──┤ Rz(1.3691) ├┤ Ry(2.2853) ├─┼─»
«     ┌────────────┐┌┴────────────┴┐ └────────────┘└────────────┘ │ »
«q_4: ┤ Ry(3.0457) ├┤ Rz(-0.46329) ├──────────────────────────────■─»
«     └────────────┘└──────────────┘                                »
«     ┌─────────────┐  ┌────────────┐               ┌────────────┐ »
«q_0: ┤ Ry(-2.0616) ├──┤ Rz(1.2156) ├──────■────────┤ Ry(3.0013) ├─»
«     ├─────────────┴┐┌┴────────────┤      │        ├────────────┤ »
«q_1: ┤ Ry(-0.28471) ├┤ Rz(-1.4324) ├──────■────────┤ Ry(2.1138) ├─»
«     ├─────────────┬┘├─────────────┤               ├────────────┤ »
«q_2: ┤ Ry(-2.9453) ├─┤ Rz(0.14396) ├──────■────────┤ Ry(2.9848) ├─»
«     ├─────────────┤ └─────────────┘      │        ├────────────┤ »
«q_3: ┤ Rz(0.13025) ├──────────────────────■────────┤ Ry(2.3166) ├─»
«     ├─────────────┤ ┌─────────────┐┌────────────┐┌┴────────────┴┐»
«q_4: ┤ Ry(-1.4091) ├─┤ Rz(-1.7904) ├┤ Ry(0.4922) ├┤ Rz(-0.89874) ├»
«     └─────────────┘ └─────────────┘└────────────┘└──────────────┘»
«      ┌────────────┐                   ┌─────────────┐
«q_0: ─┤ Rz(2.4671) ├─────────────────■─┤ Ry(-1.0931) ├
«     ┌┴────────────┤                 │ ├─────────────┤
«q_1: ┤ Rz(-2.4783) ├───────■─────────┼─┤ Ry(-1.0604) ├
«     ├─────────────┤       │         │ └┬────────────┤
«q_2: ┤ Rz(-1.9408) ├───────■─────────┼──┤ Ry(1.7846) ├
«     ├─────────────┤┌──────────────┐ │  └────────────┘
«q_3: ┤ Rz(-3.0301) ├┤ Ry(-0.68496) ├─┼────────────────
«     └─────────────┘└──────────────┘ │ ┌─────────────┐
«q_4: ────────────────────────────────■─┤ Ry(-1.1255) ├
«                                       └─────────────┘

In [ ]:
from qibo import gates, models
import qiskit.qasm2
gci_c = qibo_to_qiskit_circuit(c)
gci_c.draw()

»
q_0: ──■─────────────────────■────────────────────────────────────────────»
     ┌─┴─┐┌───────────────┐┌─┴─┐                          ┌──────────────┐»
q_1: ┤ X ├┤ Rz(-0.014721) ├┤ X ├──■────────────────────■──┤ Rz(-0.89689) ├»
     └───┘└───────────────┘└───┘┌─┴─┐┌──────────────┐┌─┴─┐└──────────────┘»
q_2: ───────────────────────────┤ X ├┤ Rz(-0.29207) ├┤ X ├───────■────────»
                                └───┘└──────────────┘└───┘     ┌─┴─┐      »
q_3: ──────────────────────────────────────────────────────────┤ X ├──────»
                                                               └───┘      »
q_4: ─────────────────────────────────────────────────────────────────────»
                                                                          »
«                                                              ┌───┐»
«q_0: ─────────────────────────────────────────────────────────┤ X ├»
«                                                              └─┬─┘»
«q_1: ───────────────────────────────────────────────────────────┼──»
«                           ┌────────────┐                       │  »
«q_2: ───────────────────■──┤ Rz(-0.541) ├───────────────────────┼──»
«     ┌───────────────┐┌─┴─┐└────────────┘                       │  »
«q_3: ┤ Rz(-0.092592) ├┤ X ├──────■─────────────────────────■────┼──»
«     └───────────────┘└───┘    ┌─┴─┐     ┌──────────────┐┌─┴─┐  │  »
«q_4: ──────────────────────────┤ X ├─────┤ Rz(-0.30111) ├┤ X ├──■──»
«                               └───┘     └──────────────┘└───┘     »
«     ┌──────────────┐┌───┐┌──────────────┐                                »
«q_0: ┤ Rz(-0.12455) ├┤ X ├┤ Rz(-0.59294) ├──■─────────────────────■───────»
«     └──────────────┘└─┬─┘└──────────────┘┌─┴─┐┌───────────────┐┌─┴─┐     »
«q_1: ──────────────────┼──────────────────┤ X ├┤ Rz(-0.014721) ├┤ X ├──■──»
«                       │                  └───┘└───────────────┘└───┘┌─┴─┐»
«q_2: ──────────────────┼─────────────────────────────────────────────┤ X ├»
«     ┌──────────────┐  │                                             └───┘»
«q_3: ┤ Rz(-0.64704) ├──┼──────────────────────────────────────────────────»
«     └──────────────┘  │  ┌──────────────┐                                »
«q_4: ──────────────────■──┤ Rz(-0.68719) ├────────────────────────────────»
«                          └──────────────┘                                »
«                                                                          »
«q_0: ─────────────────────────────────────────────────────────────────────»
«                          ┌──────────────┐  ┌────────────┐ ┌─────────────┐»
«q_1: ──────────────────■──┤ Rz(-0.89689) ├──┤ Ry(3.1246) ├─┤ Rz(-2.8363) ├»
«     ┌──────────────┐┌─┴─┐└──────────────┘  └────────────┘ └─────────────┘»
«q_2: ┤ Rz(-0.29207) ├┤ X ├───────■────────────────────────────────■───────»
«     └──────────────┘└───┘     ┌─┴─┐      ┌───────────────┐     ┌─┴─┐     »
«q_3: ──────────────────────────┤ X ├──────┤ Rz(-0.092592) ├─────┤ X ├─────»
«                               └───┘      └───────────────┘     └───┘     »
«q_4: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«                                                 ┌───┐┌──────────────┐»
«q_0: ────────────────────────────────────────────┤ X ├┤ Rz(-0.12455) ├»
«                                                 └─┬─┘└──────────────┘»
«q_1: ──────────────────────────────────────────────┼──────────────────»
«     ┌────────────┐ ┌────────────┐ ┌────────────┐  │                  »
«q_2: ┤ Rz(-0.541) ├─┤ Ry(1.8289) ├─┤ Rz(2.0549) ├──┼──────────────────»
«     └────────────┘ └────────────┘ └────────────┘  │  ┌──────────────┐»
«q_3: ──────■─────────────────────────────■─────────┼──┤ Rz(-0.64704) ├»
«         ┌─┴─┐     ┌──────────────┐    ┌─┴─┐       │  └──────────────┘»
«q_4: ────┤ X ├─────┤ Rz(-0.30111) ├────┤ X ├───────■──────────────────»
«         └───┘     └──────────────┘    └───

In [ ]:
# expectations from Qibo
energies = hamiltonian.eigenvalues()
ground_state_energy = float(energies[0])
print('Target energy: ', ground_state_energy)
vqe_energy = float(hamiltonian.expectation(vqe.circuit().state()))
print("VQE energy:", vqe_energy)
gci_loss = float(gci.loss(best_s, eo_d, db_rotation))
print("DBQA energy:", gci_loss)


Target energy:  -6.280513769031028
VQE energy: -5.605158270201547
DBQA energy: -6.232857305211463


In [ ]:
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

def xxz_hamiltonian(n, delta):
    """Returns the XXZ model Hamiltonian for n qubits and a given delta in Qiskit."""
    
    # Initialize lists to store the Pauli strings and coefficients
    pauli_strings = []
    coefficients = []

    for i in range(n):
        # XX term (X_i * X_{i+1})
        x_term = ['I'] * n
        x_term[i] = 'X'
        x_term[(i + 1)%n] = 'X'
        pauli_strings.append(''.join(x_term))
        coefficients.append(1.0)
        
        # YY term (Y_i * Y_{i+1})
        y_term = ['I'] * n
        y_term[i] = 'Y'
        y_term[(i + 1)%n] = 'Y'
        pauli_strings.append(''.join(y_term))
        coefficients.append(1.0)
        
        # ZZ term (Z_i * Z_{i+1})
        z_term = ['I'] * n
        z_term[i] = 'Z'
        z_term[(i + 1)%n] = 'Z'
        pauli_strings.append(''.join(z_term))
        coefficients.append(delta)

    # Create the SparsePauliOp object
    paulis = [Pauli(p) for p in pauli_strings]
    hamiltonian = SparsePauliOp(paulis, np.array(coefficients))
    
    return hamiltonian

# Example usage:
n_qubits = nqubits
delta_value = 0.5
hamiltonian = xxz_hamiltonian(n_qubits, delta_value)
print(hamiltonian)


SparsePauliOp(['XXIII', 'YYIII', 'ZZIII', 'IXXII', 'IYYII', 'IZZII', 'IIXXI', 'IIYYI', 'IIZZI', 'IIIXX', 'IIIYY', 'IIIZZ', 'XIIIX', 'YIIIY', 'ZIIIZ'],
              coeffs=[1. +0.j, 1. +0.j, 0.5+0.j, 1. +0.j, 1. +0.j, 0.5+0.j, 1. +0.j, 1. +0.j,
 0.5+0.j, 1. +0.j, 1. +0.j, 0.5+0.j, 1. +0.j, 1. +0.j, 0.5+0.j])


In [ ]:
import numpy as np

# Compute the eigenvalues using numpy's linear algebra module
eigenvalues, eigenvectors = np.linalg.eigh(hamiltonian.to_matrix())

print("Ground state energy of the Hamiltonian created in Qiskit:", eigenvalues[0])


Ground state energy of the Hamiltonian created in Qiskit: -6.280513769031028


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
 
# If you did not previously save your credentials, use the following line instead:
service = QiskitRuntimeService(channel="ibm_quantum", token="d7b55bcdcf222ad6aa9d85d01405645658facca7ed1cd3b304eabe6ded9bc5d90a5e4b783431c844e7a606e5206d1adbd0246b29c4d71392ae14b8b4d94e9e18")
# service = QiskitRuntimeService()
 
backend = service.least_busy(simulator=False, operational=True)
 
# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(vqe_c)
 
isa_circuit.draw( idle_wires=False)

qiskit_runtime_service.__init__:INFO:2024-08-15 12:57:21,034: Default instance: ibm-q/open/main
backend_converter.convert_to_target:INFO:2024-08-15 12:57:34,359: Gate calibration for instruction measure on qubits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126) is found in the PulseDefaults payload. However, this entry is not defined in the gate mapping of Target. This calibration is ignored.
backend_converter.convert_to_target:INFO:2024-08-15 12:57:34,360: Gate calibration for instruction u1 on qubits (0,) is found i

global phase: 3π/2
               ┌────┐    ┌─────────────┐     ┌────┐     ┌─────────────┐»
q_3 -> 44 ─────┤ √X ├────┤ Rz(0.92673) ├─────┤ √X ├─────┤ Rz(0.65638) ├»
          ┌────┴────┴───┐└────┬────┬───┘┌────┴────┴───┐ └────┬────┬───┘»
q_2 -> 45 ┤ Rz(-1.1188) ├─────┤ √X ├────┤ Rz(-2.5978) ├──────┤ √X ├────»
          └────┬────┬───┘ ┌───┴────┴───┐└────┬────┬───┘ ┌────┴────┴───┐»
q_1 -> 54 ─────┤ √X ├─────┤ Rz(3.1246) ├─────┤ √X ├─────┤ Rz(-1.8761) ├»
          ┌────┴────┴───┐ └───┬────┬───┘ ┌───┴────┴───┐ └────┬────┬───┘»
q_0 -> 64 ┤ Rz(-1.0954) ├─────┤ √X ├─────┤ Rz(-2.192) ├──────┤ √X ├────»
          ├─────────────┤     ├────┤    ┌┴────────────┴┐     ├────┤    »
q_4 -> 65 ┤ Rz(-2.5869) ├─────┤ √X ├────┤ Rz(-0.45428) ├─────┤ √X ├────»
          └─────────────┘     └────┘    └──────────────┘     └────┘    »
«                         ┌──────┐┌─────────────┐     ┌────┐     »
«q_3 -> 44 ───────────────┤0     ├┤ Rz(-1.1937) ├─────┤ √X ├─────»
«          ┌─────────────┐│  Ecr │└┬────────────┤     ├────┤     »
«q_2 -> 45 ┤ Rz(-2.0868) ├┤1     ├─┤ Rz(1.2229) ├─────┤ √X ├─────»
«          └─────────────┘├──────┤ └───┬────┬───┘┌────┴────┴────┐»
«q_1 -> 54 ───────────────┤0     ├─────┤ √X ├────┤ Rz(-0.36991) ├»
«          ┌─────────────┐│  Ecr │     ├────┤    └┬────────────┬┘»
«q_0 -> 64 ┤ Rz(0.84659) ├┤1     ├─────┤ √X ├─────┤ Rz(1.4969) ├─»
«          └┬────────────┤└──────┘     └────┘     └────────────┘ »
«q_4 -> 65 ─┤ Rz(1.6277) ├───────────────────────────────────────»
«           └────────────┘                                       »
«          ┌─────────────┐     ┌────┐      ┌────────────┐              »
«q_3 -> 44 ┤ Rz(-2.5626) ├─────┤ √X ├──────┤ Rz(2.7842) ├──────────────»
«          ├─────────────┤     ├────┤     ┌┴────────────┤   ┌──────┐   »
«q_2 -> 45 ┤ Rz(-1.3201) ├─────┤ √X ├─────┤ Rz(0.60005) ├───┤1     ├───»
«          └────┬────┬───┘┌────┴────┴───┐ └─────────────┘   │  Ecr │   »
«q_1 -> 54 ─────┤ √X ├────┤ Rz(0.17802) ├───────────────────┤0     ├───»
«               ├────┤    ├─────────────┴┐    ┌──────┐   ┌──┴──────┴──┐»
«q_0 -> 64 ─────┤ √X ├────┤ Rz(-0.29584) ├────┤0     ├───┤ Rz(-3.036) ├»
«               └────┘    └──────────────┘    │  Ecr │   ├────────────┤»
«q_4 -> 65 ───────────────────────────────────┤1     ├───┤ Rz(1.9963) ├»
«                                             └──────┘   └────────────┘»
«                                                                       »
«q_3 -> 44 ─────────────────────────────────────────────────────────────»
«          ┌─────────────┐     ┌────┐     ┌─────────────┐     ┌────┐    »
«q_2 -> 45 ┤ Rz(-1.2929) ├─────┤ √X ├─────┤ Rz(-1.6112) ├─────┤ √X ├────»
«          └────┬────┬───┘┌────┴────┴───┐ └────┬────┬───┘┌────┴────┴───┐»
«q_1 -> 54 ─────┤ √X ├────┤ Rz(-1.2099) ├──────┤ √X ├────┤ Rz(0.55445) ├»
«               ├────┤    ├─────────────┤      ├────┤    └┬────────────┤»
«q_0 -> 64 ─────┤ √X ├────┤ Rz(-0.8146) ├──────┤ √X ├─────┤ Rz(-3.069) ├»
«               ├────┤    ├─────────────┴┐     ├────┤    ┌┴────────────┤»
«q_4 -> 65 ─────┤ √X ├────┤ Rz(-0.86741) ├─────┤ √X ├────┤ Rz(-1.0041) ├»
«               └────┘    └──────────────┘     └────┘    └─────────────┘»
«                         ┌──────┐┌─────────────┐ ┌────┐┌─────────────┐ ┌────┐ »
«q_3 -> 44 ───────────────┤0     ├┤ Rz(-1.0145) ├─┤ √X ├┤ Rz(-1.0583) ├─┤ √X ├─»
«          ┌─────────────┐│  Ecr │├─────────────┤ ├────┤├─────────────┤ ├────┤ »
«q_2 -> 45 ┤ Rz(0.14063) ├┤1     ├┤ Rz(-1.0689) ├─┤ √X ├┤ Rz(-2.1375) ├─┤ √X ├─»
«          └───┬──────┬──┘└┬────┬┘└─┬──────────┬┘ ├────┤├─────────────┤ └────┘ »
«q_1 -> 54 ────┤0     ├────┤ √X ├───┤ Rz(1.17) ├──┤ √X ├┤ Rz(0.12529) ├────────»
«              │  Ecr │    ├────┤ ┌─┴──────────┴─┐├────┤├─────────────┤┌──────┐»
«q_0 -> 64 ────┤1     ├────┤ √X ├─┤ Rz(-0.48193) ├┤ √X ├┤ Rz(-3.0417) ├┤0     ├»
«              └──────┘    └────┘ └──────────────┘└────┘└─────────────┘│  Ecr │»
«q_4 -> 65 ────────────────────────────────────────────────────────────┤1     ├»
«             

In [ ]:
from qiskit.quantum_info import Statevector
state_vec_vqe = Statevector.from_instruction(vqe_c)
expectation_value_vqe = np.real(np.dot(state_vec_vqe.data.conj().T, np.dot(hamiltonian.to_matrix(), state_vec_vqe.data)))
print("VQE energy:", expectation_value_vqe)

VQE energy: -5.605158270201548


In [ ]:
state_vec_gci = Statevector.from_instruction(gci_c)
expectation_value_gci = np.real(np.dot(state_vec_gci.data.conj().T, np.dot(hamiltonian.to_matrix(), state_vec_gci.data)))
print("DBQA energy:", expectation_value_gci)

DBQA energy: -6.232857305211471


In [ ]:
print("Target energy:", ground_state_energy)

Target energy: -6.280513769031028


# Run simulation on IonQ

In [ ]:
from qiskit.circuit.library import EfficientSU2
# Plotting functions
import matplotlib.pyplot as plt

In [ ]:
# runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService(channel="ibm_quantum", token="d7b55bcdcf222ad6aa9d85d01405645658facca7ed1cd3b304eabe6ded9bc5d90a5e4b783431c844e7a606e5206d1adbd0246b29c4d71392ae14b8b4d94e9e18")
backend = service.least_busy(operational=True, simulator=False)

qiskit_runtime_service.__init__:INFO:2024-08-20 09:40:56,849: Default instance: ibm-q/open/main


In [ ]:
isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."